<a href="https://colab.research.google.com/github/jacobmrCuzzins/Machine-DeepLearning-/blob/main/Convolutional_Neural_Networks_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classifying Images Using CNN
This notebook is to explore the concepts with respect to training and evaluating classifiers such as the Convolutional Neural Networks (CNNs).

[https://keras.io/](https://keras.io/)




## Accessing Data from Google Drive
The dataset for this assignment is the CIFAR-10 dataset that can be found here:
https://www.cs.toronto.edu/~kriz/cifar.html

The CIFAR-10 and CIFAR-100 are well studied, yet challenging image recognition dataset. The CIFAR-10 has up to 10 classes to classify and contains 60,000 32x32 images. You should read the description of the dataset and download the dataset for Python, that is

CIFAR-10 python version: https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz

Once downloaded you need to then extract and upload the `cifar-10-batches-py` directory your Google Drive so that you can access it from within your Google Colab.

You can mount the Google Drive from the menu on the left or uncomment use the code below mount the drive.  See here for documentation on file access in Colab:

[External data: Local Files, Drive, Sheets, and Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Functions to work with CIFAR

The functions below help with access to the CIFAR-10 data the you have downloaded.

In [ ]:
import pickle
import numpy as np
import pandas as pd
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

def load_CIFAR_batch(filename, flatten=True, categorical=True):
    """ load single batch of cifar """
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding='bytes')
        X = datadict[b'data']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1).astype("float")
        if (flatten):
          X = X.reshape(10000, 3072)
        X = X.astype('float32')
        X /= 255

        y = datadict[b'labels']
        y = np.array(y)
        if (categorical):
          y = pd.get_dummies(y).values

        return X, y

def load_CIFAR_meta(filename):
  with open(filename,'rb') as f:
    metadict = pickle.load(f, encoding='bytes')

    class_labels = [ val.decode() for val in metadict.get(b'label_names') ]
    return class_labels

def get_image(X, index, nchans=3, size=32):
  xi = X[index,:]
  img = xi.reshape(32, 32, 3)
  return img

## Load the CIFAR data

The CIFAR data has 5 batches of data and 1 test data set. Each batch is labelled
- `data_batch_1`
- `data_batch_2`
- `data_batch_3`
- `data_batch_4`
- `data_batch_5`

and a test set labelled
- `test_batch`

each batch has 10,000 images, so 50,000 training and 10,000 test images.

Below is example of loading the first batch of training data labelled as `data_batch_1`.  You will need to update the path to match where you have stored your cifar-10 data.

LOADING ONLY ONE BATCH

In [ ]:
X, y = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1', flatten = False)

# split into train and test
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, stratify=y)

# Create 1d version for testing accuracy
ytrain_1d = np.argmax(ytrain, axis=1)
ytest_1d = np.argmax(ytest, axis=1)

print('We have {} instances of images'.format(y.shape[0]))

We have 10000 instances of images


TO LOAD THE FULL BATCH

In [ ]:
flatten = False
categorical = True

# load only the first batch
#X1, y1 = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1',flatten=flatten,categorical=categorical)

# load only the second batch
#X2, y2 = load_CIFAR_batch('/content/drive/My Drive/COMP2712/data/cifar-10-batches-py/data_batch_2',flatten=flatten,categorical=categorical)

# Load the first batch from CIFAR-10
X, y = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_1',flatten= False ,categorical=categorical)

# iterate over 2 to 5
for bi in range(2,6):
  # load the next data set 'bi'
  X2, y2 = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/data_batch_{}'.format(bi),flatten=flatten,categorical=categorical)

  # concatenate/stack the dataest together
  X = np.vstack([X, X2])
  y = np.vstack([y, y2])

 #Load the test data into test variables
Xtest, ytest = load_CIFAR_batch('/content/drive/My Drive/cifar-10-batches-py/test_batch', flatten=False)

# Create 1d version for testing accuracy
ytrain_1d = np.argmax(y, axis=1)
ytest_1d = np.argmax(ytest, axis=1)

print('We have {} instances of images'.format(y.shape[0]))


We have 50000 instances of images


The number of instances/examples for all the different classes.  There are 10 different classes.

In [ ]:
[np.sum(np.argmax(y, axis=1) == i) for i in range(0,10)]

[5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]

The labels for the classes are stored in the `batches.meta` file

In [ ]:
class_labels = load_CIFAR_meta('/content/drive/My Drive/cifar-10-batches-py/batches.meta')
print(class_labels)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


-

**BUILD THE CNN MODEL FROM ASSIGNMENT SHEET**

This is the CNN Model built from the given structure in the assignment

In [ ]:
X_flatten = X.reshape((50000, -1))

In [ ]:
print(X.shape)

(50000, 32, 32, 3)


In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization, Activation


model_cnn = Sequential()

# Add the first convolutional layer
model_cnn.add(Conv2D(32, kernel_size=(3, 3), input_shape=(32, 32, 3)))

model_cnn.add(BatchNormalization())  # Add Batch Normalization
model_cnn.add(Activation('relu'))  # Add activation function


# Add a max pooling layer
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Add a second convolutional layer
model_cnn.add(Conv2D(64, kernel_size=(3, 3)))
model_cnn.add(BatchNormalization())  # Add Batch Normalization
model_cnn.add(Activation('relu'))  # Add activation function

# Add another max pooling layer
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))

# Add a third convolutional layer
model_cnn.add(Conv2D(128, kernel_size=(3, 3)))
model_cnn.add(BatchNormalization())  # Add Batch Normalization
model_cnn.add(Activation('relu'))  # Add activation function

# Flatten the output for the dense layers
model_cnn.add(Flatten())

# Add a fully connected layer with 128 units
model_cnn.add(Dense(128, activation='relu'))

# Add a dropout layer for regularization
model_cnn.add(Dropout(0.5))

# Add the output layer for classification
model_cnn.add(Dense(10, activation='softmax'))

# Compile the model
model_cnn.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Print a summary of the model architecture
model_cnn.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 batch_normalization_3 (Bat  (None, 30, 30, 32)        128       
 chNormalization)                                                
                                                                 
 activation_2 (Activation)   (None, 30, 30, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 batch_normalization_4 (Bat  (None, 13, 13, 64)       

Improvements made in this model:

Added an additional convolutional layer to capture more complex features.
Added an additional max pooling layer to reduce spatial dimensions.
Added a dropout layer with a rate of 0.5 for regularization.
Increased the number of filters in the convolutional layers for more feature extraction capacity.
Please note that while these changes can potentially improve the performance of the model, it's important to experiment and fine-tune the architecture and hyperparameters based on your specific dataset and task. Additionally, monitor the training process and evaluate the model's performance on a separate validation set.

In [ ]:
# sklearn evaluation packages
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

**5x Stratefied K-fold for evaluation of model**

In [ ]:
# Stratified K-Fold for evaluation of generalisation performance

from sklearn.metrics import accuracy_score

k = 5 # 10 is the gold standard
kf = StratifiedKFold(n_splits=k)
foldi = 1
ac = [] # accuracy_score
cr = [] # classification_report
cm = [] # confusion_matrix
y_test_max_all = []
y_pred_max_all = []

activation = 'relu'
no_epochs = 15


for train_index, test_index in kf.split(X_flatten,ytrain_1d):

  print('Training with {0} for {1} for fold {2} or {3}'.format(activation, no_epochs, foldi, k))
  history = model_cnn.fit(X, y, epochs=no_epochs, verbose=1, validation_data=(Xtest, ytest),  # Add validation data
              callbacks=[early_stopping])  # Add callbacks)

  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch


  loss_final = np.sqrt(float(hist['loss'].tail(1)))
  print()
  print('Final Loss on training set: {}'.format(round(loss_final, 3)))

  # evaluate the model
  y_pred_real = model_cnn.predict(Xtest)
  y_pred_max = np.argmax(y_pred_real, axis=-1).astype(int)
  y_test_max = ytest.argmax(axis=1).astype(int)

  y_pred_max_all.extend(y_pred_max)
  y_test_max_all.extend(y_test_max)

  ac.append(accuracy_score(y_test_max,y_pred_max))
  print('accuracy is {:.2f}%'.format(ac[-1]*100)) # Print accuracy score
  print()
  cr.append(classification_report(y_test_max,y_pred_max)) # Print summary report
  cm.append(confusion_matrix(y_test_max, y_pred_max))

  foldi = foldi + 1

# Calculate the average accuracy and standard deviation
average_accuracy = np.mean(ac) * 100
std_dev_accuracy = np.std(ac) * 100

# Print the results
print(f'Average Accuracy: {average_accuracy:.2f}%')
print(f'Standard Deviation of Accuracy: {std_dev_accuracy:.2f}%')


print('k-fold complete!')

Training with relu for 15 for fold 1 or 5
Epoch 1/15
1563/1563 [==============================] - 10s 7ms/step - loss: 0.2285 - accuracy: 0.9201 - val_loss: 0.9953 - val_accuracy: 0.7650
Epoch 2/15
1563/1563 [==============================] - 9s 6ms/step - loss: 0.2210 - accuracy: 0.9224 - val_loss: 1.1894 - val_accuracy: 0.7478
Epoch 3/15
1563/1563 [==============================] - 10s 7ms/step - loss: 0.2129 - accuracy: 0.9265 - val_loss: 1.1285 - val_accuracy: 0.7529
Epoch 4/15
1563/1563 [==============================] - 10s 6ms/step - loss: 0.2105 - accuracy: 0.9263 - val_loss: 1.1072 - val_accuracy: 0.7505
Epoch 5/15
1563/1563 [==============================] - 10s 6ms/step - loss: 0.2055 - accuracy: 0.9281 - val_loss: 1.0418 - val_accuracy: 0.7619
Epoch 6/15
1563/1563 [==============================] - 9s 6ms/step - loss: 0.1985 - accuracy: 0.9297 - val_loss: 1.1932 - val_accuracy: 0.7508

Final Loss on training set: 0.446
313/313 [==============================] - 1s 2ms/step
